# Indexing data in Neo4j


In [8]:
#Shell command line in the jupyter
!jupyter nbconvert --to python connection.ipynb

[NbConvertApp] Converting notebook connection.ipynb to python
[NbConvertApp] Writing 217 bytes to connection.py


In [9]:
from py2neo import Graph, Node, Relationship, NodeSelector
from connection import graph
import pandas as pd
import numpy as np

## Accessing Dataset Processed

In [5]:
data = pd.read_csv('../data/dataset_Facebook_processed.csv', sep=";")
data.columns

Index(['Page total likes', 'Type', 'Category', 'Post Month', 'Post Weekday',
       'Post Hour', 'Paid', 'comment', 'like', 'share', 'Post id',
       'Increase likes'],
      dtype='object')

## Creating nodes
There are some import nodes that may be created once, for example:
* The weekdays (Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday)
* Page Likes
* Posts
* Comments
* Likes
* Shares

In [11]:
weekdaysList = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

print("Creating Weekdays...")

for i in weekdaysList:
    query  = "CREATE (n: Weekday {name: '" + i +"'}) RETURN n.name"
    result = graph.data(query)
    print(result)
    

Creating Weekdays...
[{'n.name': 'Sunday'}]
[{'n.name': 'Monday'}]
[{'n.name': 'Tuesday'}]
[{'n.name': 'Wednesday'}]
[{'n.name': 'Thursday'}]
[{'n.name': 'Friday'}]
[{'n.name': 'Saturday'}]


In [12]:
weekdaysList = ["Action","Product","Inspiration"]

print("Creating Categories...")

for i in weekdaysList:
    query  = "CREATE (n: Category {name: '" + i +"'}) RETURN n.name"
    result = graph.data(query)
    print(result)
    
    

Creating Categories...
[{'n.name': 'Action'}]
[{'n.name': 'Product'}]
[{'n.name': 'Inspiration'}]


In [14]:
print("Creating Page Likes...")
query  = "CREATE (n: PageLikes{name: 'Page Likes', total: 0}) return n.name"
result = graph.data(query)
print(result)

Creating Page Likes...
[{'n.name': 'Page Likes'}]


In [15]:
selector = NodeSelector(graph)


for i, row in data.iterrows():  
    transaction = graph.begin()    
    
    post = Node("Post",type = str(row['Type']), id = str(row['Post id']) )
    transaction.create(post)
    
    #weekday = selector.select("Weekday", name=str(row['Post Weekday']))
    #query = "MATCH (n:Category{name:'" + str(row['Category']) + "'}) return n"
    #category = graph.data(query)
    
    #relationship = Relationship(post, "IS_ABOUT", category)
    #transaction.create(relationship)
    
    #query = "MATCH (n:Weekday{name:'" + str(row['Post Weekday']) + "'}) return n"
    #weekday = graph.data(query)
    
    #relationship = Relationship(post, "POSTED_IN", weekday, month=str(row['Post Month']), hour=str(row['Post Hour']))
    #transaction.create(relationship)
    
    comment = Node("Comment", number  = str(row['comment']))
    transaction.create(comment)
    
    relationship = Relationship(comment, "HAS_COMMENTED", post)
    transaction.create(relationship)
    
    like = Node("Like", number  = str(row['like']))
    transaction.create(like)
    
    relationship = Relationship(like, "HAS_LIKED", post)
    transaction.create(relationship)
    
    share = Node("Share", number  = str(row['share']))
    transaction.create(share)
    
    relationship = Relationship(share, "HAS_SHARED", post)
    transaction.create(relationship)
    
    transaction.commit()


In [16]:
from pandas import DataFrame
transaction = graph.begin()
query = "MATCH (n:Category{name:'Inspiration'}) return n.name"
cate  = graph.data(query)
query = "MATCH (n:Post{id:'12'}) return n.id"
n = graph.data(query)
relationship = Relationship(n, "IS_ABOUT", cate)
transaction.create(relationship)
transaction.commit()

In [ ]:
query = "MATCH (n:Category{name:'" + str(row['Category']) + "'}) return n.id"
category = graph.data(query)
category


In [ ]:
selector = NodeSelector(graph)
selected = selector.select("Weekday", name="Saturday")
print(selected)